# Natural Language Processing

## *Workshop 4*  [![Open In Colab](https://github.com/oballinger/QM2/blob/main/colab-badge.png?raw=1)](https://colab.research.google.com/github/oballinger/QM2/blob/main/notebooks/W04.%20Natural%20Language%20Processing.ipynb)


## Background

Exxon Mobil is the 4th largest oil company in the world. In 1978, an Exxon scientist named James Black wrote an [internal briefing](https://insideclimatenews.org/documents/james-black-1977-presentation/) called "The Greenhouse Effect" in which he warned: “Present thinking holds that man has a time window of five to ten years before the need for hard decisions regarding changes in energy strategies might become critical.”

Rather than acting on this information, Exxon spent the next [forty years aggressively funding climate denial](https://news.harvard.edu/gazette/story/2021/09/oil-companies-discourage-climate-action-study-says/). Recently, [a U.S. court ruled](https://www.theguardian.com/environment/2022/may/24/exxon-trial-climate-crimes-fossil-fuels-global-heating) that ExxonMobil must face trial over accusations that it lied about the climate crisis and covered up the fossil fuel industry’s role in worsening environmental devastation.

### Earnings Calls
Every three months, Exxon conducts an ["earnings call"](https://www.investopedia.com/terms/e/earnings-call.asp); a conference call between the management of a public company, analysts, investors, and the media to discuss the company’s financial results during a given reporting period, such as a quarter or a fiscal year.

You can [register](https://globalmeet.webcasts.com/starthere.jsp?ei=1488251&tp_key=440e363aaf) to attend their next one if you want! No worries if you miss it, they provide [transcripts](https://corporate.exxonmobil.com/Investors/Investor-relations/Investor-materials-archive#Quarterlyearningsmaterials) on their website.

These transcripts provide an intimate window into the company's dealings. We can see how much pressure investors are putting on the company to tackle climate change, and how the company responds.

We'll be working with transcripts spanning nealry 20 years and over 10 million words; that's like reading the Harry Potter series 10 times. Then, we'll look at a sample of 100,000 tweets that use the #ExxonKnew hashtag, and analyze public pressure on the company.

I gathered the data used in this lab for two papers:

* [Transition, Hedge, or Resist? Understanding Political and Economic Behavior toward
Decarbonization in the Oil and Gas Industry](https://discovery.ucl.ac.uk/id/eprint/10196075/1/ssrn-3694447.pdf)

* [Using Earnings Calls to Understand the Political Behavior of Major Polluters
](https://muse.jhu.edu/article/847625)

![](https://haha.business/business.jpg)


## Downloading the Data
Let's grab the data we will need this week from our course website and save it into our data folder. If you've not already created a data folder then do so using the following command.

Don't worry if it generates an error, that means you've already got a data folder.

In [1]:
%%capture
!pip install spacy
!pip install scattertext
!pip install tika
!pip install spacytextblob
!python -m spacy download en_core_web_sm

In [2]:
import spacy
import json
import pylab
from IPython.core.display import display, HTML
import nltk
from tika import parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spacytextblob.spacytextblob import SpacyTextBlob

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10., 8.)
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')

## Downloading and reading one earnings call

Exxon host earnings calls on their website in PDF form. Usually, working with PDFs is a real pain as they are not machine-readable. Using a python package called [tika](https://www.geeksforgeeks.org/parsing-pdfs-in-python-with-tika/), we can "parse" a pdf, turning it into machine-readable text:

In [3]:
# define the URL where your PDF lives. You could also upload your own pdf.
#url='https://corporate.exxonmobil.com/-/media/Global/Files/investor-relations/quarterly-earnings/earnings-transcripts/2022-earnings-transcripts/1Q22-XOM-Earnings-Call-Transcript-4-29-22.pdf'
url='https://d1io3yog0oux5.cloudfront.net/_74d009918ead0ec6acdd6bbaf27a8316/exxonmobil/db/2288/22123/earnings_release/XOM+2Q23+Earnings+Press+Release+Website.pdf'
# parse the pdf by feeding tika the URL and store the text in an object called "raw"
raw = parser.from_file(url)


2025-01-04 20:23:16,599 [MainThread  ] [INFO ]  Retrieving https://d1io3yog0oux5.cloudfront.net/_74d009918ead0ec6acdd6bbaf27a8316/exxonmobil/db/2288/22123/earnings_release/XOM+2Q23+Earnings+Press+Release+Website.pdf to /tmp/_74d009918ead0ec6acdd6bbaf27a8316-exxonmobil-db-2288-22123-earnings_release-xom-2q23-earnings-press-release-website.pdf.
INFO:tika.tika:Retrieving https://d1io3yog0oux5.cloudfront.net/_74d009918ead0ec6acdd6bbaf27a8316/exxonmobil/db/2288/22123/earnings_release/XOM+2Q23+Earnings+Press+Release+Website.pdf to /tmp/_74d009918ead0ec6acdd6bbaf27a8316-exxonmobil-db-2288-22123-earnings_release-xom-2q23-earnings-press-release-website.pdf.
2025-01-04 20:23:16,895 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2

RuntimeError: Unable to start Tika server.

Now, we have an object called "raw" that contains some useful information. Notice the squiggly brackets; this is a dictionary. It contains several fields, including some useful metadata such as the author

In [ ]:
raw['metadata']

In [ ]:
date=raw['metadata']['dcterms:created']
title=raw['metadata']['dc:title']
raw_text=raw['content']

print('Date: ', date)
print('Title: ', title)
print('Word Count: ', len(raw_text))
print('Text:')
raw_text

look at that! we're beginning to give some structure to our text data. But suppose I wanted to analyze multiple earnings calls; I need to organize this data so that it can accomodate new entries. As always, we want to **tabularize** our data. Let's create a dataframe with three columns (Date, Title, and Text) in which each row is one earnings call:

In [ ]:
# create a dataframe using the above data
call=pd.DataFrame({'Date':[date],'Title':[title],'Text':[raw_text]})

# remember, datetime information almost always reaches us as text.
# we need to explicitly convert it to the datetime data type.
call['Date']=pd.to_datetime(call['Date'])

# Let's see what we've got.
call

Now, if we were so inclined, we could use a loop to repeat this process for a large number of earnings calls, yielding a neatly organized dataframe containing the date, title, and text of earnings calls over time. I've done this so you don't have to, and stored it as a file called "Exxon.json". It spans 2002-2019, and contains over 10 million words' worth of earnings calls. Let's take a peek:

In [ ]:
df=pd.read_json('https://storage.googleapis.com/qm2/wk4/Exxon.json')
df

Great-- we've got a structured dataset of earnings calls. But even though the data has *structure*, the data in the "Text" column still needs some cleaning and processing.

## Dirty Words

Text often comes 'unclean' either containing tags such as HTML (or XML), or has other issues.
We've already done a bit of tidying, but it's been relatively straightforward. Be cautious when committing to a text analysis project - you may spend a great deal of time tidying up your text.

For example, you may have noticed "\n\n\n\n\n\n\n\n..." in the text of the first earnings call we downloaded. This is a character (just like "a" or "$") except it indicates that we want to create a new line. It's part of the formatting of the pdf. That's not really useful information to us. Let start by selecting an earnings call; i've chosen the 38th in this dataframe:

In [ ]:
call=df.iloc[38]

print('Date: ', call['Date'])
print('Title: ', call['Title'])
print('Word Count: ', len(call['Text']))
print('Text:')
call['Text']

This call took place on May 25th, 2016. The transcript is over 125,000 words, nearly as long as the third Lord of the Rings book. It would be a pain to read all of it, so we'll use python to extract insights. Currently, the contents of `call["Text"]` is a ["string"](https://docs.python.org/3/library/stdtypes.html#text-sequence-type-str)-- a sequence of characters. We can do a number of things with strings, including splitting a big string into smaller strings using a specific delimiter and the `.split()` function. For example, I can break down the whole text of the earnings call roughly into sentences by splitting the string every time I encounter a period ("."). This returns a list of smaller strings, and if i select the first one using `[0]`, I get the first sentence of this call:

In [ ]:
call['Text'].split('.')[0]

Lovely! the first sentence is an introduction by then-CEO [Rex Tillerson](https://en.wikipedia.org/wiki/Rex_Tillerson).

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Rex_Tillerson_official_portrait.jpg/800px-Rex_Tillerson_official_portrait.jpg.jpg" alt="drawing" width="200"/>

He was CEO of Exxon from 2006 until he retired on January 1st 2017. One month later, he was sworn in as U.S. Secretary of State under Donald Trump. Let's see what Rex thinks about climate change!

## Regular Expressions (Regex)

Another thing we can do with strings in python is search them using regular expressions. A regular expression is a sequence of characters that specifies a search pattern in text. You can play around building some regex queries using this [tool](https://regexr.com/).

You can think about this as Ctrl+F on steroids; In its simplest form, we can use regex to search for a character, word, or phrase in a bunch of text. For example, we can use regular expressions to count how many times "climate change" is mentioned in this earnings call using the `re.findall()` function:

In [ ]:
# import the regular expressions library
import re

# use the findall function to search for mentions of "climate change" in the text of our call
climate_change = re.findall(r'climate change', call['Text'], re.IGNORECASE)

# this returns a list of strings matching our search term.
# the length of the list gives us the number of occurances
len(climate_change)

Looks like climate change is mentioned 51 times in this earnings call.



---

### Exercise

how many times is the phrase "global warming" mentioned?

---



But we have 182 earnings calls in this sample-- suppose we want to count the number of times climate change is mentioned in each one, so we can see the salience of this topic over time.

### Applying a lambda function to a dataframe

Because each row of our dataframe `df` is an earnings call (the text of which is contained in `df['Text']`, we want to apply the analysis we did for the single earnings call above to each row of `df`.

We can accomplish this using a [**lambda function**](https://www.w3schools.com/python/python_lambda.asp). This allows us to iterate over each value in a dataframe column, and apply a function to it. In the simple example below, I use a lambda function to create a new column that is takes the values from a different column and multiplies them by 2:

In [ ]:
# create a dataframe called "example" with one column called "numbers" which contains numbers 0-5
example= pd.DataFrame({'numbers':[0,1,2,3,4,5]})

# print the dataframe
print("\n Before applying lambda function: \n", example)

# create a new column called "doubled numbers"
# apply a lambda function that iterates over each row in the "numbers" column
# call each row "x", and multiply it by 2
example['doubled numbers']= example['numbers'].apply(lambda x: x*2)

# print the dataframe, which now contains the new column
print("\n \n After applying lambda function: \n", example)

There were simpler ways of doing this (namely, `example[doubled numbers]=example['numbers']*2`). But if we want to do something more complex, lambda functions are very useful. Remember, we used `re.findall(r'climate change', call['Text'], re.IGNORECASE)` to get a list of mentions of climate change in the text of one earnings call, and measured the length of the list using `len()` to count the number of mentions. We can turn this into a lambda function as follows:

`df['Text'].apply(lambda x: len(re.findall(r'climate change', x, re.IGNORECASE)))`

1. `df['Text']`: the column we want to iterate over.
2. `.apply(lambda x:`: iterate over each row in the column, and call each value in that column x. In other words, x will represent the text of each earnings call.
3. `len(re.findall(r'climate change', x, re.IGNORECASE)` this is exactly the same as what did previously to find the number of mentions of climate change in the one earnings call, except that we swapped `call['Text']` with `x`, since we want to do this for the text of every earnings call.

In [ ]:
# create a column called "climate change" that contains the count of mentions of this keyword
df['climate change']=df['Text'].apply(lambda x: len(re.findall(r'climate change', x, re.IGNORECASE)))

# print the title of each earnings call, along with the number of mentions of climate change.
print(df[['Title','climate change']])

Amazing! We've now got a column indicating how many times "climate change" was mentioned in each earnings call.  

---

### Exercise

Create three new columns that count the frequency of the terms "global warming", "carbon capture", and another phrase or word of your choosing. When you've done this, edit the code below so that it not only shows the frequency of "climate change" mentions, but also the three additional columns you created.

Advanced: Our current measure of the number of mentions of keywords might be biased: if one earnings call mentions climate change 10 times more than another, but that earnings call has 10 times more words, then the *rate* of keyword mentions hasn't actually increased; people are just talking more. You can get the word count of each earnings call in the lambda function above using `len(x)` (using `len()` on a string will get you a word count). Edit the lambda function above such that we don't get a *count* of the number of mentions of climate change, but the *rate* of mentions (i.e., count of "climate change" divided by total word count per call).


---


Let's plot the frequency of these mentions over time to analyze temporal trends in the salience of climate change and other keywords in these calls. We'll accomplish this using

In [ ]:
# extract the year from the date column
df['Year']=df['Date'].dt.year

# group the dataframe by year, calculating the sum of the "climate change" column
# save it as a new dataframe called "yearly"
yearly=df.groupby('Year')['climate change'].sum()

# plot yearly
yearly.plot()

Do you notice any patterns in the salience of these topics over time?

## Intermediate Regex

Great. We can see how frequently climate related keywords come up in earnings calls between shareholders and Exxon Mobil representatives over time. But what if we want to look at what they're actually saying?

We can get a bit fancier with Regex to look at the content of these discussions. Regex can be pretty confusing, but it's also a very powerful tool. Before moving on, let's familiarize ourselves a bit more with regex.

Let's try to extract all *sentences* containing the phrase "climate change"; the regex would look like this:

`([^.]*climate change[^.]*)`

1. `()` indicates that we want to match a group of characters, not just the characters themselves. In this case, the group is a sentence, not just the word climate change. But how do we
2. `[^.]*` we want to match all characters except periods. This will break the text up into sentences
3. `climate change` the phrase we want our sentence to contain.

when you put it all together, the regex will search for groups of characters (1.) bounded by periods (2.) that contain the phrase "climate change" (3.)

In [ ]:
# create a list called "climate_sentences" that contains the results of this query
climate_sentences=re.findall(r"([^.]*climate change[^.]*)"," ".join(df['Text']))

print(len(climate_sentences))
# print the first 10 sentences in the list
for sentence in climate_sentences[:10]:
  print('\n', sentence)

### Semantic Analysis

Now we can see the *sentences* which mention climate change, which helps us understand a bit about the context. We can perform semantic analysis on some of these sentences to take a close look at the grammar of some of these sentences; I've isolated the 9th sentence and produced a dependency tree, like the ones we've seen in class.

In [ ]:
from spacy import displacy

#run the NLP pipeline on the 9th sentence from our list of sentences about climate change.
doc = nlp(climate_sentences[8].lstrip())

#dataframe to store the results
results=pd.DataFrame()

for token in doc:
    # create a row for each token in the sentence, containing the text, lemma, Part Of Speech, and dependency relation
    row=pd.DataFrame({'text':[token.text], 'lemma':[token.lemma_], 'pos':[token.pos_], 'dep':[token.dep_]})
    # append the row to the results dataframe
    results=pd.concat([results, row])

# print the results
results


In [ ]:

#print out the dependency tree
displacy.render(doc, jupyter=True)

The root of this dependency tree is the verb "describes". The main subject is the Economist, and the object is Exxon. But we're still missing one vital piece of information: who is speaking? It makes a big difference to our understanding of whats going on. Are mentions of climate change increasing over time because shareholders are asking more questions? Or did CEO Rex Tillerson have a spiritual awakening in which all he wants to do is talk about climate change? For that, we need to figure out who's talking, and resturcture our dataframe.

## Advanced Regex

The earnings call transcript is structured in such a way that it should be possible to separate speakers based on regular expressions. Every time a new person is speaking, they are introduced in the transcript in a new paragraph; Consider the excerpt below:

```
OPERATOR: Our next question comes from Philip Weiss with Argus Research.

PHILIP WEISS, ANALYST, ARGUS RESEARCH COMPANY: Good morning. I did have one, most of my questions have been answered, but I do have one follow-up on the US. You said that the rig count that's being used for liquids-rich is rising but when I look at production, natural gas as a percentage of your total production has grown, and liquids has actually fallen a little bit. So, I wonder if you can just comment on when we might start to see that trend change?

DAVID ROSENTHAL: Sure. The fall off in the liquids is really just the overall decline in the conventional, as well as some divestments. You'll recall we had a divestment in the Eastern Gulf of Mexico and that had an impact on us year-over-year in particularly in the second half.
In terms of when we'll see significant production growth out of the unconventional, I mentioned some of the increases in percentages, although we haven't given all of the specific production volumes, but we'll do that as we progress.
```

Now, we can't simply split by new line (`\n`); David Rosenthal has two paragraphs. We also can't just split using `:`, since this may appear in the text other than to indicate speakers. Let's describe the features of the characters we're looking to split out:

`([A-Z]+.+[A-Z]+:)`

  1. It's a group of characters
    * regex: `()`
  2. The words are all caps, and can contain any characters
    * regex:`([A-Z])`
  2. There can be multiple words, and they can be separated by anything
    * regex: `([A-Z]+.+[A-Z])`
  4. The sequence always ends in a colon
    * regex: `([A-Z]+.+[A-Z]+:)`

Let's use this regex in `re.findall()` to get a list of the speakers on this call:

In [ ]:
# create a list of all the speakers by searching text of the earnings call for the above regex.
speakers = re.findall(r'([A-Z]+.+[A-Z]+: )', call['Text'])

# because they don't introduce the speaker in the opening statement, insert a placeholder at the beginning of this list.
speakers.insert(0,'INTRODUCTION')

# using set(list) will give you the unique values in a list
# the length of set(list) gives us the number of unique speakers
print('There are', len(set(speakers)),'speakers on this call:')

# let's print out the first 10 speakers:
for speaker in speakers[:10]:
  print(speaker)

We want to do more than just identify the speakers though; we want to break up the text of our earnings call into chunks of speech and associate each chunk of speech with its speaker. We can split a string using regular expressions using `re.split(<regex>,<text>)`. This takes one block of text, splits it into chunks using the regex, and returns a list of chunks:

In [ ]:
# split the text of the earnings call using our regex, save the list as "speech"
speech=re.split(r'[A-Z]+.+[A-Z]+: ', call['Text'])

# now, we can print the fourth speaker:
print('Speaker: \n', speakers[3])

# and the text of the fourth speech:
print('\n Speech: \n', speech[3])

Now we've associated chunk of speech with their speaker, amazing. Let's create a new dataframe that reflects this structure. Currently, our dataframe `call` has one row. Let's use the two lists we just created, `speakers` and `speech`, to create a dataframe in which each row is one chunk of speech. A column called "speaker" will indicate who is speaking, and a column called "speech" will contain the text of the speech:

In [ ]:
# create the new dataframe, from the two lists, and name it "speaker_df"
speaker_df=pd.DataFrame({"speaker":speakers,"speech":speech})

# clean up the "speaker" column by removing the colons using .str.replace(":","")
# remove trailing white space using str.rstrip()
speaker_df['speaker']=speaker_df['speaker'].str.replace(':','').str.rstrip()

# print rows in which rex tillerson is speaking:
print(speaker_df[speaker_df['speaker']=="REX TILLERSON"])

## Analyzing Distinguishing Terms

And there we have it. We started with a PDF on a website, and we've ended up with a dataframe in which each row is a speech, with a column indicating who is speaking, what they're saying, and when they said it.

Now, lets use this dataframe to create a scatterplot comparing the language used by the company's CEO Rex Tillerson and the company's shareholders.This will give us insights into the topics that are important for shareholders, and the debates that take place within the company.

We'll do so using the [scattertext](https://spacy.io/universe/project/scattertext) libray:

In [ ]:
%%capture

import scattertext as st

# create a corpus of text from the dataframe
corpus = st.CorpusFromPandas(speaker_df, # load the dataframe
                             category_col='speaker', # indicate which column contains the category we want to distinguish by
                             text_col='speech', # indicate which column stores the text to be analyzed
                             nlp=nlp).build() # load the NLP models used for analysis

# remove stopwords from the corpus of text
corpus=corpus.remove_terms(nlp.Defaults.stop_words, ignore_absences=True)

# now, we create the scatterplot
html = st.produce_scattertext_explorer(
                   corpus, # load the corpus
                   category="REX TILLERSON", # indicate which category value we want to compare against all others; in this case, all rows in which "REX TILLERSON" is the speaker
                   category_name='Rex Tillerson', # set the label on the plot as "Rex Tillerson"
                   not_category_name='Others', # set the label on the plot for all other speakers as "Others"
                   width_in_pixels=1000) #set the width

In [ ]:
# display the plot
display(HTML(html))

The plot above compares the frequency of terms used by Exxon CEO Rex Tillerson (on the Y axis) against those used by other speakers (mainly shareholders, on the X axis). The top right corner will contain terms used frequently by both groups. the bottom left corner contains terms used infrequently by both groups. The top left corner contains terms used frequently by Rex Tillerson, but infrequently by shareholders. The bottom right corner contains terms used frequently by shareholders, but infrequently by Rex Tillerson.

A list of top terms used by each group is shown in the right. On this list, we can see that "climate change" is the 4th most common phrase used by the "Others" category, but isn't even in the top ten for Rex. If you click on "climate change" in this list, it will give you some statistics on how frequently this term is used by each group, as well as a selection of example sentences in which the term appears. You can search for other words/phrases either by clicking on them in the scatterplot, or entering them into the "Search the chart" box below the scatterplot.


---


### Exercise

1. Search for "ALEC" in this plot, and then google the term to find out more about what this is. What are shareholders aiming to do regarding ALEC, and how does Tillerson respond?

2. Search for the term "carbon". What differences do you notice in the use of this term by Rex Tillerson versus the shareholders?

3. Use this plot to identify another topic that shareholders are pressuring Exxon about.


---



# External pressure

The fact that Exxon knew about climate change in the 1970s and still funded climate denial resulted in public outrage, culminating in an online movement organized around the twitter hashtag #ExxonKnew.

I've downloaded almost 100,000 tweets containing the hashtag #ExxonKnew, between 2016 and 2017. Work together as a group to explore and analyze this dataset.

In [6]:
tweets=pd.read_csv('https://storage.googleapis.com/qm2/wk4/Exxon_tweets_clean.csv')
tweets=tweets.sample(frac=1, random_state=1) #don't edit this line, i'm shuffling the data for you
tweets

,text,created_at,author_id,lang,latitude,longitude
13362,RT @Exxon_Knew: ICYMI: @LamarSmithTX21 keeps p...,2016-11-13T07:09:30.000Z,36152170,en,NaN,NaN
45804,Look what Exxon has bought and paid for... #Ex...,2016-06-17T15:46:41.000Z,31419872,en,NaN,NaN
59678,RT @350: It's time to make fossil fuel compani...,2016-05-07T15:29:06.000Z,1576283767,en,NaN,NaN
62120,"RT @Energydesk: ""There is no doubt"": 1980 docs...",2016-04-27T08:49:56.000Z,429464827,en,NaN,NaN
8046,RT @MarkRuffalo: Why does Putin want Trump Pre...,2016-12-11T02:33:34.000Z,56839488,en,NaN,NaN
...,...,...,...,...,...,...
21440,RT @greenpeaceusa: This is a big deal. #ExxonK...,2016-09-20T18:01:48.000Z,7.15E+17,en,NaN,NaN
73349,the @FBI is investigating what #ExxonKnew abou...,2016-03-29T15:53:32.000Z,750681121,en,NaN,NaN
50057,RT @sierraclub: Exxon knew about #climatechang...,2016-05-29T06:39:38.000Z,461795005,en,NaN,NaN
5192,"RT @PriceofOil: #ExxonKnew, Exxon lied. And no...",2016-12-13T23:03:07.000Z,14247309,en,NaN,NaN


### Sentiment Analysis

Sentiment analysis is the computational study of people's opinions, sentiments, emotions, appraisals, and attitudes towards entities such as products, services, organizations, individuals, issues, events, topics, and their attributes. Let's study the sentiment of the tweets in this dataset.

[spacytextblob](https://spacytextblob.netlify.app/) performs sentiment analysis using the TextBlob library. Adding spacytextblob to a spaCy nlp pipeline creates a new extension attribute for the Doc.

The `._.blob` attribute contains all of the methods and attributes that belong to the `textblob.TextBlob` class. Some of the common methods and attributes include:

1. `._.blob.polarity`: a float within the range [-1.0, 1.0].
2. `._.blob.subjectivity`: a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
3. `._.blob.sentiment_assessments.assessments`: a list of polarity and subjectivity scores for the assessed tokens.

Let's run sentiment analysis on a single tweet:

In [26]:
# grab the 130th row in the dataframe and select the text of the tweet
text=tweets.iloc[130]['text']

# apply the NLP pipeline to this text.
doc = nlp(text)

print(text)
print('Polarity: ', doc._.blob.polarity)
print('Subjectivity: ', doc._.blob.subjectivity)
print('Assessments: ', doc._.blob.sentiment_assessments.assessments)


RT @simonrlomax: #Climate Witch Hunt: The Left Is Coming for You Next https://t.co/O7ant6fsgt
https://t.co/O7ant6fsgt via .@KevinNR @NRO #E…
Polarity:  0.0
Subjectivity:  0.0
Assessments:  [(['left'], 0.0, 0.0, None), (['next'], 0.0, 0.0, None)]


We can see that the model has deemed this tweet to be expressing negative sentiment: it has a polarity of -0.15. It also deems this to be a pretty subjective tweet, with a subjectivity score of 0.8. It does indeed appear to be expressing a subjective opinion. Finally, we can see which words are leading to this assessment. The word "good" is leading to a 0.7 increase in the polarity score, and a 0.6 increase in the subjectivity score. The word "worst" is leading to a -1 change polarity, and a +1 change in subjectivity. The overall scores are weighted averages of these values. Though these scores do roughly align with the actual sentiment of this tweet, **ALWAYS** pay attention to whats going on inside of your sentiment analysis pipeline. Even though the overall sentiment score here is negative, it should probably be even more negative; the algorithm picked up on the word "good" in this tweet, and this improved the polarity score by 0.7. But the context in which "good" was uttered in this tweet is actually negative! the person is saying "stop saying #Tillerson is good on climate"-- this is expressing negative sentiment!


### Assessed Question

In this assessed question, we want to use NLP and spatial analysis to map out where the people who are most angry about Exxon are located.

In the code cell below:

1. create a dataframe called "sample" which contains the first 3000 tweets from the "tweets" dataframe.
2. Using a lambda function, `.apply(lambda x: nlp(x)._.blob.polarity)`, create a column in the sample dataframe that contains the polarity of each tweet.
3. Create a column that contains the subjectivity score for each tweet.
4. Filter the dataframe to keep only the tweets that are subjective (subjectivity score > 0.5) *and* tweets that have negative sentiment (polarity score < 0).

In [20]:
# question 1
sample = tweets.head(3000)
print("\before applying lambda function",sample)
# Print column names of a dataframe
print(sample.columns)

sample['polarity']  = sample['text'].apply(lambda x: nlp(x)._.blob.polarity)
sample ['subjectivity'] = sample['text'].apply(lambda x: nlp(x)._.blob.subjectivity)
filtered_sample = sample [(sample['subjectivity'] > 0.5) & (sample['polarity'] < 0)]
filtered_sample






efore applying lambda function                                                     text  \
13362  RT @Exxon_Knew: ICYMI: @LamarSmithTX21 keeps p...   
45804  Look what Exxon has bought and paid for... #Ex...   
59678  RT @350: It's time to make fossil fuel compani...   
62120  RT @Energydesk: "There is no doubt": 1980 docs...   
8046   RT @MarkRuffalo: Why does Putin want Trump Pre...   
...                                                  ...   
20781  @Heritage Challenge in court, Congressional su...   
25921  Tell @theAGU, the biggest org of Earth scienti...   
82601  #ExxonKnew now @NatlAssnAttysGn can follow @AG...   
83425  Sign here to tell @jiminhofe &amp; his @EPWMaj...   
77080  Exxon lied about an existential threat to huma...   

                     created_at   author_id lang  latitude  longitude  
13362  2016-11-13T07:09:30.000Z    36152170   en       NaN        NaN  
45804  2016-06-17T15:46:41.000Z    31419872   en       NaN        NaN  
59678  2016-05-07T15:29:06.000Z

<ipython-input-20-bb08e2672fe7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['polarity']  = sample['text'].apply(lambda x: nlp(x)._.blob.polarity)
<ipython-input-20-bb08e2672fe7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample ['subjectivity'] = sample['text'].apply(lambda x: nlp(x)._.blob.subjectivity)


,text,created_at,author_id,lang,latitude,longitude,polarity,subjectivity
77813,"RT @350: Wow, this is bad news for Big Oil: th...",2016-03-05T16:59:59.000Z,1976624676,en,NaN,NaN,-0.200000,0.588889
17177,RT @Exxon_Knew: NEW REPORT: @exxonmobil's clim...,2016-10-09T06:26:41.000Z,195610523,en,NaN,NaN,-0.431818,0.727273
8081,RT @Exxon_Knew: Rex Tillerson for Sec of State...,2016-12-11T02:23:30.000Z,120969408,en,NaN,NaN,-0.100000,0.600000
20425,"RT @350: ""The coral reef of Elliot Key, Florid...",2016-09-22T17:38:10.000Z,118553423,en,NaN,NaN,-0.100000,0.700000
78638,RT @AndreaLeon: #Exxon Mobil's insane argument...,2016-02-28T17:59:36.000Z,2528569476,en,NaN,NaN,-0.450000,0.550000
...,...,...,...,...,...,...,...,...
1104,RT @Exxon_Knew: Stop saying #Tillerson is good...,2016-12-24T00:35:06.000Z,15792803,en,NaN,NaN,-0.150000,0.800000
5489,Today Trump officially nominates Exxon's CEO t...,2016-12-13T15:56:07.000Z,2890411277,en,NaN,NaN,-0.700000,0.666667
33700,LA Times paying price for poor journalism as #...,2016-08-05T18:02:05.000Z,356478638,en,NaN,NaN,-0.400000,0.600000
32530,RT @ClimateTruthOrg: World's biggest climate s...,2016-08-11T15:58:55.000Z,55908595,en,NaN,NaN,-0.416667,0.666667


In [27]:
# Group by 'author_id' and sum the polarity
author = filtered_sample.groupby('author_id', as_index=False).sum()

# Sort by polarity in ascending order and select the user with the lowest polarity
author = author.sort_values(by='polarity', ascending=True)

author


,author_id,text,created_at,lang,latitude,longitude,polarity,subjectivity
117,406723394,RT @ClimateTruthOrg: Disgusted by Exxon-funded...,2016-07-20T05:06:04.000Z2016-07-20T05:07:00.000Z,enen,0.0,0.0,-1.600000,1.900000
12,14266598,There's dark money behind the attack on New Yo...,2016-10-21T15:16:18.000Z2016-04-18T22:05:41.00...,enenen,0.0,0.0,-1.337879,2.318182
158,7.15E+17,RT @RepTedLieu: I commend the SEC for investig...,2016-09-21T15:26:15.000Z2016-10-14T20:38:55.000Z,enen,0.0,0.0,-1.100000,1.600000
19,156772034,RT @EnergyInDepth: .@AGSchneiderman's pathetic...,2016-08-22T14:59:46.000Z2016-08-09T11:59:01.00...,enenen,0.0,0.0,-1.081818,2.177273
182,89011980,RT @ClimateTruthOrg: Disgusted by Exxon-funded...,2016-07-14T00:21:08.000Z,en,0.0,0.0,-1.000000,1.000000
...,...,...,...,...,...,...,...,...
173,764061452,RT @commondreams: #SEC Probe Called Possible '...,2016-09-21T01:38:38.000Z,en,0.0,0.0,-0.050000,0.800000
49,229237604,RT @commondreams: #SEC Probe Called Possible '...,2016-09-20T21:13:31.000Z,en,0.0,0.0,-0.050000,0.800000
82,2878645527,RT @ALECexposed: Exxon's #Tillerson could be n...,2016-12-12T19:14:20.000Z,en,0.0,0.0,-0.050000,0.800000
162,7.25E+17,RT @bobbywego: 1999 #ClimateJustice analysis S...,2016-05-10T01:38:31.000Z,en,0.0,0.0,-0.050000,0.950000


*Question: which twitter user hates Exxon the most (lowest total

---

sentiment)? Make note of their ID.*

@ClimateTruthOrg Author ID:  406723394 hates Exxon the most with lowest total sentiment

#GIRLIEE!!! DO NOT SUBMIT WITHOUT FIGURING THIS NOTE OUT:
#Look up becuase does total sentiment mean sum of all the polarity of one particular user or does it just relate to a singular tweets subjectivity and polarity scores